# Attempt 2

In [ ]:
import math
import time
import torch
import gpytorch
import numpy as np
import random as rnd

from matplotlib import pyplot as plt

# randomizes the datapoint inputs every run
np.random.seed(2) 
# np.random.seed(rnd.randint(0, 1000000)) # deactivated the randomizer for simplicity during testing

# timer module to calculate run time
time_start = time.time() 

# Stage 0: Setting up HF and LF data sets

In [ ]:
# Training data is 1000 points in [0,1] inclusive regularly spaced
X = torch.linspace(0, 1, 1000)

# defines the HF and LF sine functions. Gaussian noise was excluded for simplicity during testing
def hf(x): 
    return 1.8*torch.sin(x * (8 * math.pi))*2*x # + torch.randn(x.size()) * math.sqrt(0.04) #* the accurately simulated sin model for HF data

def lf(x):
    return torch.sin(x * (8 * math.pi))*x # + torch.randn(x.size()) * math.sqrt(0.04) #* the inaccurately simulated sin model for LF data

# Setting the number of LF and HF points
Nlf=20
Nhf=8

#Sampling LF and HF model randomly
X_lf = np.random.permutation(X)[0:Nlf] # basically randomly aranges the 50 different values in X array, from 0 to "len(Nlf)"
X_hf = np.random.permutation(X_lf)[0:Nhf]

# Converting np.ndarray to tensor to be able to utilize GPyTorch
X_lf = torch.from_numpy(X_lf)
X_hf = torch.from_numpy(X_hf) 

# Stage 1: HF Modelling using Exact Gaussian Process Model
Step 1: Setting up the HF data

In [ ]:
#Labeling train_x and train_y data
train_x = X_hf # Randomly arranged HF points
train_y = hf(X_hf) # HF points ran through HF sine function (accurate sine function)

Step 2: Setting up the HF Model

In [ ]:
# Simplest form of Gaussian processs model, Exact intereference
class ExactGPModel(gpytorch.models.ExactGP): # gpytorch.models.ExactGP is the simplest GP Model.
    def __init__(self, train_x, train_y, likelihood): 
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood) # Adds the training sets and likelihood to the GP model
        self.mean_module = gpytorch.means.ConstantMean() #  A Mean - This defines the prior mean of the GP.
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) # A Kernel - This defines the prior covariance of the GP-
        # a method of using a linear classifier to solve a non-linear problem.

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x) # This is the object used to represent multivariate normal distributions.
        # multivariate Gaussian distribution, or joint normal distribution is a generalization of the one-dimensional normal distribution to higher dimensions. 

Step 3: Initializing the model and likelihood and smoke testing

In [ ]:
# set a number of training iterations for both models 
training_iter = 300

# initialize likelihood and model 
likelihood = gpytorch.likelihoods.GaussianLikelihood() #? how likekly a model has a certain parameter value given some data
model = ExactGPModel(train_x, train_y, likelihood) # calling ExactGPModel from the class as "model"
# print(model)

# this is for running the notebook in our testing framework with an added smoke test 
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else training_iter

Step 4: Training the model

In [ ]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward() # Computes the gradient and stores them in the tensors once you call .backward() 
    
    # print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
    #     i + 1, training_iter, loss.item(),
    #     model.covar_module.base_kernel.lengthscale.item(),
    #     model.likelihood.noise.item()
    # ))
    optimizer.step()

Step 5: Make predictions with the model

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var(): # turns off gradients computation
    test_x = torch.linspace(0, 1, 1000)
    observed_pred = likelihood(model(test_x)) # output of LF model with the likelihoodand model
print(observed_pred)

# Stage 2: LF Model Modelling using Exact Gaussian Process Model

In [ ]:
# Set the low fidelity data values

train_x2 = X_lf # Randomly arranged HF points
train_y2 = lf(X_lf)  # LF points ran through LF sine function (inaccurate sine function)

In [ ]:
# Simplest form of Gaussian processs model, Exact intereference
class ExactGPModel_LF(gpytorch.models.ExactGP): 
    def __init__(self, train_x2, train_y2, likelihood): 
        super(ExactGPModel, self).__init__(train_x2, train_y2, likelihood) 
        self.mean_module = gpytorch.means.ConstantMean() 
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) 

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# initialize likelihood and model 
likelihood = gpytorch.likelihoods.GaussianLikelihood() 
model_2 = ExactGPModel(train_x2, train_y2, likelihood) 

import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else training_iter

In [ ]:
# train the model and likelihood
model_2.train()
likelihood.train()

optimizer = torch.optim.Adam(model_2.parameters(), lr=0.1) 

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_2)

for i in range(training_iter):
    optimizer.zero_grad()

    output = model_2(train_x2)

    loss = -mll(output, train_y2)

    loss.backward()

    optimizer.step()

In [ ]:
model_2.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_x2 = torch.linspace(0, 1, 1000)
    observed_pred2 = likelihood(model_2(test_x2))

# Stage 3: MF Model Stage 
I might need this code box below later on

In [ ]:
"""
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module1 = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        self.covar_module2 = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        # We learn an IndexKernel for 2 tasks
        # (so we'll actually learn 2x2=4 tasks with correlations)
        self.task_covar_module1 = TwoFidelityIndexKernel(num_tasks=2, rank=1)
        self.task_covar_module2 = TwoFidelityIndexKernel(num_tasks=2, rank=1, includeParams=False)
        print(f"Initial value : Covar 1, lengthscale {self.covar_module1.base_kernel.lengthscale.item()}, prefactor {self.covar_module1.outputscale.item()}")
        print(f"Initial value : Covar 2, lengthscale {self.covar_module2.base_kernel.lengthscale.item()}, prefactor {self.covar_module2.outputscale.item()}")
        
    def forward(self,x,i):
        mean_x = self.mean_module(x)
        
        # Get input-input covariance
        covar1_x = self.covar_module1(x)
        # Get task-task covariance
        covar1_i = self.task_covar_module1(i)
        
        # Get input-input covariance
        covar2_x = self.covar_module2(x)
        # Get task-task covariance
        covar2_i = self.task_covar_module2(i)
        
            
        # Multiply the two together to get the covariance we want
        covar1 = covar1_x.mul(covar1_i)
        covar2 = covar2_x.mul(covar2_i)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar1+covar2)
"""

In [ ]:
# Total failure here since best I can do is try to mix X_hf and L_hf inputs within the model
# instead of using the LF model and predict using HF inputs
# then using those values to add into MF model

train_x3 = X_lf
train_y3 = lf(X_lf)

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP): 
    def __init__(self, hf_p, train_3, likelihood): 
        super(ExactGPModel, self).__init__(hf_p, train_y3, likelihood) 
        self.mean_module = gpytorch.means.ConstantMean() 
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel()) 

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x) 

In [ ]:
training_iter = 300

hf_p = X_hf
hf_p = torch.linspace(min(hf_p), max(hf_p), 20)

likelihood = gpytorch.likelihoods.GaussianLikelihood() 
model_3 = ExactGPModel(hf_p, train_y3, likelihood)

import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else training_iter

# Stage 4: Plotting

In [ ]:
with torch.no_grad():
    f, ax = plt.subplots(3, figsize = (10, 5))

    # Get upper and lower confidence bounds
    lower,  upper  =  observed_pred.confidence_region()
    lower2, upper2 = observed_pred2.confidence_region()
    lower3, upper3 = observed_pred3.confidence_region()
    
    #High Fidelity / Exact Line
    ax[0].plot(X,hf(X), label="High Fidelity / Exact")
    ax[1].plot(X,hf(X), label="High Fidelity / Exact")
    ax[2].plot(X,hf(X), label="High Fidelity / Exact")

    # Sample Points
    ax[0].plot(train_x.numpy(),train_y.numpy(), 'ro',label="High Fidelity Samples")
    ax[1].plot(train_x2.numpy(),train_y2.numpy(), 'bo', label="Low Fidelity Samples")
    
    # GP Mean PREDICTIONS
    ax[0].plot(test_x.numpy(),observed_pred.mean.numpy(), 'k', label = "HF GP Mean (Trained on Red Dots)")
    ax[1].plot(test_x2.numpy(),observed_pred2.mean.numpy(), 'k', label = "LF GP Mean (Train on Blue Dots)")
    ax[2].plot(test_x3.numpy(),observed_pred3.mean.numpy(), 'k', label = "Deep GP Mean (Train on Blue Dots)")
    
    # Shade between the lower and upper confidence bounds
    ax[0].fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.2, label = "Confidence")
    ax[1].fill_between(test_x2.numpy(), lower2.numpy(), upper2.numpy(), alpha=0.2, label = "Confidence") 
    ax[2].fill_between(test_x3.numpy(), lower3.numpy(), upper3.numpy(), alpha=0.2, label = "Confidence") 

    #Legends
    ax[2].legend(bbox_to_anchor=(0.9, 1), loc='upper left', fontsize='x-small')
    ax[1].legend(bbox_to_anchor=(0.9, 1), loc='upper left', fontsize='x-small')
    ax[0].legend(bbox_to_anchor=(0.9, 1), loc='upper left', fontsize='x-small')

abs_path = os.path.abspath('')
dir = os.path.dirname(abs_path)

plt.savefig(abs_path + "\\attempt_2_plot.pdf")
print(f"Finished in {(time.time() - time_start)} seconds." )